# rl2.ipynb

This notebook contains the second step in the 5-step process outlined in November 13th, 2020, in order to produce an actor-critic architecture for pendulum control.

In step 1 we create a network with N units that takes the activity of 1 unit and transforms it into a bell-shaped representation.
In here we put 2 of those as the inputs to a (N/2)x(N/2) grid, and visualize.

To achieve bell-shaped tuning, we use the `bell_shaped_1D` unit model.

In [1]:
%cd ../..
import numpy as np
import matplotlib.pyplot as plt
import time
from draculab import *

/home/z/projects/draculab/notebook


In [2]:
N = 10 # number of units in each S network
No2 = int(np.ceil(N/2))
net_params = {'min_delay' : 0.005,
              'min_buff_size' : 10 }
A_params = {'type' : unit_types.source,
            'init_val' : 0.5,
            'function' : lambda t: None}
L_params = {'type' : unit_types.sigmoidal,
            'thresh' : 0.5,
            'slope' : 4.,
            'tau' : 0.02,
            'init_val' : 0.5 }
L_geom = {'shape' : 'sheet',
          'arrangement' : 'grid',
          'rows' : No2,
          'columns' : No2,
          'center' : [0., 0.],
          'extent' : [1., 1.] }
S_params = {'type' : unit_types.bell_shaped_1D,
            'init_val' : 0.1,
            'tau' : 0.01,
            'center' : list(np.linspace(0., 1., N)),
            'b' : 30. }
S1_params = S_params.copy()
S2_params = S_params.copy()
S1_params['coordinates'] = [np.array([-.6, -.5 + i/N]) for i in range(N)]
S2_params['coordinates'] = [np.array([-.5 + i/N, -.5]) for i in range(N)]

# S1_geom = {'shape' : 'sheet',
#            'arrangement' : 'grid',
#            'rows' : N,
#            'columns' : 1,
#            'center' : [-1.5, 0.],
#            'extent' : [.5, 1.] }
# S2_geom = {'shape' : 'sheet',
#            'arrangement' : 'grid',
#            'rows' : 1,
#            'columns' : N,
#            'center' : [0., -1.5],
#            'extent' : [1., .5] }


net = network(net_params)
topo = topology()
A = net.create(2, A_params)
S1 = net.create(N, S1_params)
S2 = net.create(N, S2_params)
L = topo.create_group(net, L_geom, L_params)

net.units[A[0]].set_function(lambda t: 0.5*(np.sin(t)+1.))
net.units[A[1]].set_function(lambda t: 0.5*(np.cos(t)+1.))

A__S_conn = {'rule' : 'all_to_all',
             'delay' : 0.01 }
A__S_syn = {'type' : synapse_types.static,
            'init_w' : 1. }
S1__L_conn_spec = {'connection_type' : 'divergent',
                  'mask' : {'circular' : {'radius' : 2. }},
                  'kernel' : 1.,
                  'delays' : {'linear' : {'c' : 0.01, 'a': 0.01}},
                  'weights' : {'gaussian' : {'w_center' : 1., 'sigma' : 0.1}},
                  'dist_dim' : 'y',
                  'edge_wrap' : True,
                  'boundary' : {'center' : [-0.05, 0.], 'extent':[1.1, 1.]} }
S2__L_conn_spec = S1__L_conn_spec.copy()
S2__L_conn_spec['dist_dim'] = 'x'
S2__L_conn_spec['boundary'] = {'center' : [0., -0.05], 'extent':[1., 1.1]}
S1__L_syn_spec = {'type' : synapse_types.static }
S2__L_syn_spec = {'type' : synapse_types.static }

net.connect([A[0]], S1, A__S_conn, A__S_syn)
net.connect([A[1]], S2, A__S_conn, A__S_syn)
topo.topo_connect(net, S1, L, S1__L_conn_spec, S1__L_syn_spec)
topo.topo_connect(net, S2, L, S2__L_conn_spec, S2__L_syn_spec)

In [5]:
start_time = time.time()
#times, activs, _ = net.run(10.)
times, activs, _ = net.flat_run(10.)
print('Execution time is %s seconds' % (time.time() - start_time))
activs = np.array(activs)

Execution time is 4.789627313613892 seconds


In [6]:
fs = (20,6)
A_fig = plt.figure(figsize=fs)
A_activs = np.array(activs[A])
plt.plot(times, A_activs.transpose())
plt.title('A')

S1_fig = plt.figure(figsize=fs)
S1_activs = np.array(activs[S1])
plt.plot(times, S1_activs.transpose())
S1_legends = [str(ctr) for ctr in S1_params['center']]
plt.legend(S1_legends)
plt.title('S1')

S2_fig = plt.figure(figsize=fs)
S2_activs = np.array(activs[S2])
plt.plot(times, S2_activs.transpose())
S2_legends = [str(ctr) for ctr in S2_params['center']]
plt.legend(S2_legends)
plt.title('S2')

plt.show()

In [11]:
len(activs[0])

2000

In [13]:
%matplotlib qt5
# values from act_anim are not reaching update_act_anim because I got rid of 'self'

def act_anim(pop, thr, interv=100, slider=False):
    """ An animation to visualize the activity of a set of units. 

        pop : an array or list with the IDs of the units to visualize.
        interv : refresh interval of the simulation.
        slider : When set to True the animation is substituted by a slider widget.
        thr : units whose activity surpasses 'thr' will be highligthted.
    """
    unit_fig = plt.figure(figsize=(10,10))
    ax = unit_fig.add_axes([0., 0., 1., 1.], frameon=False)
    xcoords = [ u.coordinates[0] for u in [net.units[i] for i in pop] ]
    ycoords = [ u.coordinates[1] for u in [net.units[i] for i in pop] ]
    min_x = min(xcoords) - 0.1;     max_x = max(xcoords) + 0.1
    min_y = min(ycoords) - 0.1;     max_y = max(ycoords) + 0.1
    ax.set_xlim(min_x, max_x), ax.set_xticks([])
    ax.set_ylim(min_y, max_y), ax.set_yticks([])
    scat = ax.scatter(xcoords, ycoords, s=20.*activs[pop,0])
    n_data = len(activs[0])
    act_thr = thr
    act_anim_pop = pop
    
    animation = FuncAnimation(unit_fig, update_act_anim,
                            interval=interv, save_count=int(round(net.sim_time/net.min_delay)))
    return animation

def update_act_anim(frame):
    # Each frame advances one simulation step (min_delay time units)
    idx = frame%n_data
    cur_time = net.min_delay*idx
    scat.set_sizes(300.*activs[act_anim_pop,idx])
    scat.set_color(color_fun(activs[act_anim_pop, idx]))
    unit_fig.suptitle('Time: ' + '{:f}'.format(cur_time))
    return ax,

act_anim(L, thr=0.1)

Traceback (most recent call last):
  File "/home/z/.conda/envs/draculab/lib/python3.8/site-packages/matplotlib/cbook/__init__.py", line 224, in process
    func(*args, **kwargs)
  File "/home/z/.conda/envs/draculab/lib/python3.8/site-packages/matplotlib/animation.py", line 959, in _start
    self._init_draw()
  File "/home/z/.conda/envs/draculab/lib/python3.8/site-packages/matplotlib/animation.py", line 1703, in _init_draw
    self._draw_frame(next(self.new_frame_seq()))
  File "/home/z/.conda/envs/draculab/lib/python3.8/site-packages/matplotlib/animation.py", line 1726, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "<ipython-input-13-54c1d9ad2a68>", line 31, in update_act_anim
    scat.set_sizes(300.*activs[act_anim_pop,idx])
NameError: name 'scat' is not defined


In [4]:
# Visualization for connections

# visualizing the connections 
%matplotlib qt5
# The 'notebook' backend is compatible with FuncAnimation
from matplotlib.animation import FuncAnimation
    
sourceIDs = S2
sinkIDs = L
# the update function below will use these variables
all_coords = [u.coordinates for u in [net.units[uid] for uid in sourceIDs]]
source = sourceIDs #[u.ID for u in net.units]
sink = sinkIDs # [u.ID for u in net.units]
len_source = len(sourceIDs)
len_sink = len(sinkIDs)
source_0 = sourceIDs[0]
sink_0 = sinkIDs[0]

# flattening net.syns
all_syns = []
for syn_list in [net.syns[i] for i in sinkIDs]:
    all_syns.extend([s for s in syn_list if s.preID in sourceIDs])

# getting lists with the coordinates of all source, sink units
source_coords = [u.coordinates for u in [net.units[i] for i in sourceIDs]]
sink_coords = [u.coordinates for u in [net.units[i] for i in sinkIDs]]
source_x = [c[0] for c in source_coords]
source_y = [c[1] for c in source_coords]
sink_x = [c[0] for c in sink_coords]
sink_y = [c[1] for c in sink_coords]

# id2src[n] maps the unit with network id 'n' to its index in the 'source' list
id2src = np.array([1e8 for _ in range(len(net.units))], dtype=int) # 1e8 if not in source
for src_idx, net_idx in enumerate(sourceIDs):
    id2src[net_idx] = src_idx
# id2snk[n] maps the unit with network id 'n' to its index in the 'sink' list
id2snk = np.array([1e8 for _ in range(len(net.units))], dtype=int) # 1e8 if not in sink
for snk_idx, net_idx in enumerate(sinkIDs):
    id2snk[net_idx] = snk_idx

# setting colors
std_src = [0., 0.5, 0., 0.5]
std_snk = [0.5, 0., 0., 0.5]
big_src = [0., 0., 1., 1.]
big_snk = [0., 0., 1., 1.]

# constructing figure, axes, path collections
conn_fig = plt.figure(figsize=(10,6))
ax1 = conn_fig.add_axes([0.02, 0.01, .47, 0.95], frameon=True, aspect=1)
ax2 = conn_fig.add_axes([0.51, 0.01, .47, 0.95], frameon=True, aspect=1)
src_col1 = ax1.scatter(source_x, source_y, s=2, c=std_src)
snk_col1 = ax1.scatter(sink_x, sink_y, s=2, c=std_snk)
src_col2 = ax2.scatter(source_x, source_y, s=2, c=std_src)
snk_col2 = ax2.scatter(sink_x, sink_y, s=2, c=std_snk)
ax1.set_title('sent connections')
ax2.set_title('received connections')
ax2.set_yticks([])

# preparing to print weights
w_mat = np.zeros((len(sinkIDs), len(sourceIDs)))
for syn in all_syns: 
    w_mat[id2snk[syn.postID], id2src[syn.preID]] = abs(syn.w)
w_mat /= np.amax(w_mat) # normalizing (maximum is 1)
cmap = plt.get_cmap('Reds') # getting colormap

# At each frame we'll visualize the connections arising from a single unit
def update(frame): 
    sou_u = frame%len_source # source unit whose receivers we'll visualize
    snk_u = frame%len_sink # sink unit whose senders we'll visualize

    # PLOTTING THE RECEIVERS OF sou_u ON THE LEFT AXIS
    source_sizes = np.tile(2, len_source)
    sink_sizes = 2. + 400.*w_mat[:,sou_u]
    source_colors = np.tile(std_src,(len_source,1))
    sink_colors = cmap.__call__(w_mat[:,sou_u])
    source_sizes[sou_u] = 320
    source_colors[sou_u] = big_src
    src_col1.set_sizes(source_sizes)
    snk_col1.set_sizes(sink_sizes)
    snk_col1.set_color(sink_colors)
    src_col1.set_color(source_colors)

    # PLOTTING THE SENDERS TO snk_u ON THE RIGHT AXIS
    source_sizes = 2. + 400.*w_mat[snk_u,:]
    sink_sizes = np.tile(2, len_sink)
    source_colors = cmap.__call__(w_mat[snk_u,:])
    sink_colors = np.tile(std_snk, (len_sink,1))
    sink_sizes[snk_u] = 30
    sink_colors[snk_u] = big_snk
    src_col2.set_sizes(source_sizes)
    snk_col2.set_sizes(sink_sizes)
    src_col2.set_color(source_colors)
    snk_col2.set_color(sink_colors)

    return ax1, ax2,

animation = FuncAnimation(conn_fig, update, interval=250, blit=True)
plt.show()

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB o

In [ ]:
len(net.units)